In [ ]:
# XGboost 원리 : 약한 나무로 나온 오차로 다음 나무가 학습.    -- 데이터가 불균형할때 쓰기 좋음
# 랜덤포레스트 : 랜덤한 나무 100개

# 약한 학습기 : 성능이 안좋은 모델 (찍은것보단 나은 수준의 모델)  -  일부러 성능 안좋은 모델 여러개 만들어서  다 더하면 원하는 모델을 얻을 수 있게

In [2]:
import pandas as pd
bike = pd.read_csv("Data/bike data.csv")
del bike["datetime"]
bike.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,1,0,0,1,9.84,14.395,81,0.0,16
1,1,0,0,1,9.02,13.635,80,0.0,40
2,1,0,0,1,9.02,13.635,80,0.0,32
3,1,0,0,1,9.84,14.395,75,0.0,13
4,1,0,0,1,9.84,14.395,75,0.0,1


In [12]:
# 문제집,정답지 만들기
X = bike.drop("count", axis=1).values
Y = bike["count"].values

# 데이터 분할
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, Y)

In [13]:
from sklearn.tree import DecisionTreeRegressor

model1 = DecisionTreeRegressor(max_depth=2)      # 성능을 일부러 낮게 망가뜨림 
model1.fit(train_x, train_y)         # 성능이 안좋은 약한 학습기로 학습

train_y2 = train_y - model1.predict(train_x)       # 첫번째모델의 오차를 train_y2에 저장
train_y2

array([ 45.8042394 , -90.49453322,  61.21985472, ...,  17.8042394 ,
       189.71877518,  19.21985472])

In [14]:
model2 = DecisionTreeRegressor(max_depth=2)
model2.fit(train_x, train_y2)         # 첫번째모델이 틀린만큼 학습  =>  첫 번째 모델이 틀린만큼을 맞춰주려고

train_y3 = train_y2 - model2.predict(train_x)      # 두번째모델의 오차를 train_y3에 저장
train_y3

array([ 40.34429116, -95.95448146, 101.38990209, ...,  12.34429116,
       184.25882694,  59.38990209])

In [15]:
model3 = DecisionTreeRegressor(max_depth=2)
model3.fit(train_x, train_y3)         # 첫번째모델이 틀린만큼 학습  =>  두 번째 모델이 또 틀린만큼을 세 번째 모델이 보정

DecisionTreeRegressor(max_depth=2)

In [18]:
from sklearn.metrics import mean_squared_error

pred1 = model1.predict(test_x)
pred2 = model2.predict(test_x)
pred3 = model3.predict(test_x)

pred = pred1 + pred2 + pred3

# 성능평가 - RMSE (예측값이 실제값에서 평균적으로 얼마나 떨어져 있는지)
mean_squared_error(test_y, pred) ** (1/2)

158.6201411331724

In [21]:
# XGboost (앞서 햇던 과정을 함수로 할수잇음)

from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(n_estimators=3, max_depth=2, learning_rate=1)      # 나무는 3개만, 각각의 나무의 max_depth는 2. 
                                                                                     # 학습률 1로 각 나무가 예측한 값을 그대로 더함
model.fit(train_x, train_y)

pred = model.predict(test_x)

mean_squared_error(test_y, pred) ** (1/2)

158.6201411331723

# xgboost 라이브러리로

In [34]:
# xgboost 라이브러리로 하려면 설치해서 써야됨 
# !pip install xgboost

from xgboost import XGBClassifier, XGBRegressor

# iris 데이터로 성능 비교
import seaborn as sns
iris = sns.load_dataset("iris")

X = iris.drop("species", axis = 1)
Y = iris['species']

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, Y)

In [37]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=3, max_depth=2, learning_rate=1)

model.fit(train_x, train_y)

pred = model.predict(test_x)
model.score(test_x, test_y)

0.9210526315789473

In [38]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=3, max_depth=2, learning_rate=1)

model.fit(train_x, train_y)

pred = model.predict(test_x)
model.score(test_x, test_y)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2], got ['setosa' 'versicolor' 'virginica']